In [1]:
import numpy as np
import pandas as pd
import spacy
nlp = spacy.load("ru_core_news_sm")
import tensorflow as tf
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc, mean_squared_error
from sklearn.model_selection import train_test_split
from json import load


In [7]:
not_discriptors = list()
with open('../../Data/Words/word_pairs_not_disc.txt', 'r', encoding='utf-8') as txt:
    for i in txt:
        not_discriptors.append(i.replace('\n', ''))

with open('../../Data/Events/type_of_events.json', 'r', encoding='utf-8') as js:
    events = load(js)

with open('../../Data/Datasets/data.json', 'r', encoding='utf-8') as js:
    discriptors = load(js)


In [8]:
dict_of_disc = {'expession': [], 'is_discr': []}
for subject in discriptors.values():
    for d in sum(subject.values(), []):
        dict_of_disc['expession'].append(nlp(d).vector)
        dict_of_disc['is_discr'].append(True)

for subject in not_discriptors:
    dict_of_disc['expession'].append(nlp(subject).vector)
    dict_of_disc['is_discr'].append(False)



In [13]:
features = np.array(dict_of_disc['expession'])
labels = np.array(list(map(int, dict_of_disc['is_discr'])))


In [5]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=71)


In [6]:
import tensorflow as tf

model_class = tf.keras.models.Sequential()
model_class.add(tf.keras.layers.Dense(int(X_train[0].shape[0]*1.5), activation='softmax',
                                      input_shape=(X_train[0].shape[0],), name='input'))
model_class.add(tf.keras.layers.Dense(
    int(X_train[0].shape[0]*2), activation='relu', name='hidden_layer_2'))
model_class.add(tf.keras.layers.Dense(
    int(X_train[0].shape[0]*1.5), activation='relu', name='hidden_layer_3'))
model_class.add(tf.keras.layers.Dense(
    int(X_train[0].shape[0]*0.5), activation='relu', name='hidden_layer_4'))
model_class.add(tf.keras.layers.Dense(1, name='output'))
model_class.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Dense)               (None, 144)               13968     
                                                                 
 hidden_layer_2 (Dense)      (None, 192)               27840     
                                                                 
 hidden_layer_3 (Dense)      (None, 144)               27792     
                                                                 
 hidden_layer_4 (Dense)      (None, 48)                6960      
                                                                 
 output (Dense)              (None, 1)                 49        
                                                                 
Total params: 76,609
Trainable params: 76,609
Non-trainable params: 0
_________________________________________________________________


In [7]:
model_class.compile(
    loss='mse',  # Функция потерь
    optimizer='Adam',  # Оптимизатор
    metrics=[  # Метрики
        'mse',  # Если у объекта назначено имя, то можно вызвать объект с его помощью
        # tf.keras.metrics.Precision()
    ]
)


In [8]:
model_class.fit(
    X_train,  # Набор входных данных
    y_train,  # Набор правильных ответов
    validation_split=0.4,  # Этот параметр автоматически выделит часть обучающего набора на валидационные данные. В данном случа 20%
    epochs=40,  # Процесс обучения завершится после 10 эпох
    # Набор данных будет разбит на пакеты (батчи) по 8 элементов набора в каждом.
    batch_size=2
)


Epoch 1/40
238/238 [==============================] - 4s 3ms/step - loss: 0.1503 - mse: 0.1503 - val_loss: 0.0750 - val_mse: 0.0750
Epoch 2/40
238/238 [==============================] - 0s 2ms/step - loss: 0.0491 - mse: 0.0491 - val_loss: 0.0583 - val_mse: 0.0583
Epoch 3/40
238/238 [==============================] - 1s 2ms/step - loss: 0.0321 - mse: 0.0321 - val_loss: 0.0434 - val_mse: 0.0434
Epoch 4/40
238/238 [==============================] - 1s 2ms/step - loss: 0.0241 - mse: 0.0241 - val_loss: 0.0451 - val_mse: 0.0451
Epoch 5/40
238/238 [==============================] - 1s 3ms/step - loss: 0.0211 - mse: 0.0211 - val_loss: 0.0536 - val_mse: 0.0536
Epoch 6/40
238/238 [==============================] - 0s 2ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.0368 - val_mse: 0.0368
Epoch 7/40
238/238 [==============================] - 1s 2ms/step - loss: 0.0070 - mse: 0.0070 - val_loss: 0.0384 - val_mse: 0.0384
Epoch 8/40
238/238 [==============================] - 0s 2ms/step - loss: 0.

In [9]:
predict = model_class.predict(X_test)


9/9 [==============================] - 0s 2ms/step


In [10]:
mean_squared_error(predict, y_test)


0.0580643455770958

In [11]:
predictA = model_class.predict(features)
print('Accuracy')
print(accuracy_score(labels, predictA > 0.5))
print('Confusion matrix')
print(confusion_matrix(labels, predictA > 0.5))
print('Precision, Recall, F\n', classification_report(
    labels, predictA > 0.5))


33/33 [==============================] - 0s 1ms/step
Accuracy
0.96875
Confusion matrix
[[327  22]
 [ 11 696]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.97      0.94      0.95       349
           1       0.97      0.98      0.98       707

    accuracy                           0.97      1056
   macro avg       0.97      0.96      0.96      1056
weighted avg       0.97      0.97      0.97      1056



In [12]:
print('Accuracy')
print(accuracy_score(y_test, predict > 0.5))
print('Confusion matrix')
print(confusion_matrix(y_test, predict > 0.5))
print('Precision, Recall, F\n', classification_report(
    y_test, predict > 0.5))


Accuracy
0.9242424242424242
Confusion matrix
[[ 86  12]
 [  8 158]]
Precision, Recall, F
               precision    recall  f1-score   support

           0       0.91      0.88      0.90        98
           1       0.93      0.95      0.94       166

    accuracy                           0.92       264
   macro avg       0.92      0.91      0.92       264
weighted avg       0.92      0.92      0.92       264



In [13]:
model_class.save('../../PipeLines/Classifications/checker_is_discriptor_spacy_vectorize.h5')
